In [80]:
import json
import glob
import csv
import collections

In [81]:
# ----------------------------------reading annotation files
def read_ann_file(file):
    f = open(file,"r",encoding="utf8")
    ann_Text = f.read()
    return ann_Text

In [82]:
# ------------------------------------reading labels  files
file ="data/labels.tsv"
labels = open(file,"r",encoding="utf8").read()

def get_conf_bias(file):
    for item in labels.split("\n"):
        if file in item :
            confirmation_bias = item.strip().split()
            if confirmation_bias[1] == "negative":
                return("false")
            elif confirmation_bias[1] =="positive":
                return("true")

In [83]:
#-------------------------------------getting paragraph text and anotation status
def get_Paragraphs(essay_no):
    with open("data/data-tokenized.tsv") as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
        for row in reader:
            Paragraph_dict = {"text": "" , "sufficient" : True}
            if row['ESSAY'] == str(essay_no):
                Paragraph_dict['text'] = row['TEXT']
                if row['ANNOTATION'] == 'insufficient':
                      Paragraph_dict['sufficient'] = False
                elif row['ANNOTATION'] == '' :
                      Paragraph_dict['sufficient'] = True
            unified_file.append(Paragraph_dict)

In [84]:
# -----------------------reading text files
def get_text(file):
    essay_Text =open(file,"r",encoding="utf8").read()
#     print(text)
    return essay_Text   

In [101]:
unified_file = []
ann_list = []
# confirmation_dict = {"confirmation_bias" : True }
counter = 1
# dict_para = {'text':"",'sufficient' : True }

for file in glob.glob("data/brat-project-final/essay*.ann"):

#------------------------------------ Declaring variables -------------------------------
    id_dict ={'id': 0 , "text" : ""}
    id_dict["major_claim"] = []
#     major_claim_dict = {'span' : 0 , "text": ""}
    id_dict["claims"] = []
#     claims_dict = {'span' : 0 , "text": ""}
    id_dict["premises"] =[]
#     premises_dict = {'span' : 0 , "text": ""}
    id_dict["paragraphs"] = []
    
#------------------------------------ Declaring variables Ends -------------------------------
    ann_text = read_ann_file(file)
    ann_file_name = file
    txt_file_name = ann_file_name.replace("ann","txt")
    id_dict['id'] = counter
    id_dict['text'] = get_text(txt_file_name)
    for item in ann_text.split("\n"):  # Elements collection from each Annotation file.
        major_claim_dict = {'span' : 0 , "text": ""}
        claims_dict = {'span' : 0 , "text": ""}
        premises_dict = {'span' : 0 , "text": ""}
        # ---------------------------- Collecting Major claim--------------------------------
        if "MajorClaim" in item :
            ann_list = item.strip()
            ann_list_words = ann_list.split()
            MajorClaim=' '.join(ann_list_words[4:])
            MC_Span = ann_list_words[2:4]
            MC_Span_int = list(map(int, MC_Span))
            major_claim_dict['span'] = MC_Span_int
            major_claim_dict['text'] = MajorClaim
            id_dict['major_claim'].append(major_claim_dict) 
            # ---------------------------- Collecting Claims----------------------------------
        if "Claim" in item and "MajorClaim" not in item :
            ann_claim_list = item.strip()
            ann_claim_words = ann_claim_list.split()
            Claim=' '.join(ann_claim_words[4:])
            C_Span = list(map(int,ann_claim_words[2:4]))
            claims_dict['span'] = C_Span
            claims_dict['text'] = Claim
            id_dict['claims'].append(claims_dict)
             # ---------------------------- Collecting Claims----------------------------------
        if "Premise" in item :
            ann_premise_list = item.strip()
            ann_premise_words = ann_premise_list.split()
            Premise_text=' '.join(ann_premise_words[4:])
            P_Span = list(map(int,ann_premise_words[2:4]))
            premises_dict['span'] = P_Span
            premises_dict['text'] = Premise_text
            id_dict['premises'].append(premises_dict)   
    #--------------------------exiting the for loop for single file parsing --------------------------    
    
        # -----------getting the confirmation bias. ------------------------
    conf_file_name=file.replace("data/brat-project-final\essay","essay").replace(".ann","")
    confirmation_bias = get_conf_bias(conf_file_name)
    id_dict['confirmation_bias'] = confirmation_bias
    # -----------getting the paragraphs------------------------
    
    with open("data/data-tokenized.tsv") as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
        for row in reader:
            Paragraph_dict = {"text": "" , "sufficient" : True}
            if row['ESSAY'] == str(counter):
                Paragraph_dict['text'] = row['TEXT']
                if row['ANNOTATION'] == 'insufficient':
                    Paragraph_dict['sufficient'] = False
                    id_dict['paragraphs'].append(Paragraph_dict)
                elif row['ANNOTATION'] == '' :
                    Paragraph_dict['sufficient'] = True
                    id_dict['paragraphs'].append(Paragraph_dict)
            

    unified_file.append(id_dict)
    with open("just_id.json", "w") as outfile:
        json.dump(unified_file, outfile, indent=4)
    counter = counter+1  